#  Rozwiązywanie układu równań liniowych metodami rozkładu LU

#### Podaj zasadę działania metod opartych o dekompozycję LU.
Dekompozycja LU jest jedną  zmetod rozwiązywania równań liniowych. Polega na rozbicu macierzy współczynników $A$ na 2 macierze takie, że $A = LU$, gdzie $L$ jest macierzą trójkątna dolną a $U$ macierzą trójkątną górną.

## Zadanie 1

Zaimplementuj metody rozkładu LU:
- Rozkład Crouta 
- Rozkład Doolitla
- Rozkład Choleskyego

Dla każdej z metod podaj warunki niezbędne aby można ją było zastosować. Sprawdź poprawność działania tych metod.
Przetestuj wydajność algorytmów dla kilku rozmiarów macierzy (podobnie jak w ćwiczeniu 9)

In [50]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time

Aby zastosować rozkład Crouta, na diagonali macierzy trójkątnej górnej $U$ muszą występować wyłacznie 1 ($u_{ii} = 1$).

In [51]:
def crout(A):
    n = len(A)
    L = [[0] * n for _ in range(n)]
    U = [[0] * n for _ in range(n)]
    for j in range(n):
        U[j][j] = 1

        for i in range(j, n):
            alpha = float(A[i][j])
            for k in range(j):
                alpha -= L[i][k]*U[k][j]
            L[i][j] = alpha

        for i in range(j+1, n):
            uu = float(A[j][i])
            for k in range(j):
                uu -= L[j][k]*U[k][i]

            if int(L[j][j]) == 0:
                raise ZeroDivisionError("0 occurred on diagonal. Could not compute U")

            U[j][i] = uu/L[j][j]

    return L, U

Aby zastosować rozkład Dollittle'a, na diagonali macierzy trójkątnej dolnej $L$ muszą występować wyłącznie 1 ($l_{ii} = 1$).

In [52]:
def doolittle(A):
    n = len(A)
    L = [[0] * n for _ in range(n)]
    U = [[0] * n for _ in range(n)]

    for j in range(n):
        L[j][j] = 1
        for i in range(j + 1):
            s1 = 0
            for k in range(i):
                s1 += U[k][j] * L[i][k]
            U[i][j] = A[i][j] - s1
        for i in range(j, n):
            s2 = 0
            for k in range(j):
                s2 += U[k][j] * L[i][k]
            L[i][j] = (A[i][j] - s2) / U[j][j]

    return L, U

Aby zastosować rozkład Cholesky'ego, wartości na diagonali macierzy trójkątnej dolnej $L$ oraz diagonali macierzy trójkątnej górnej $U$  muszą być sobie równe ($u_{ii} = l_{ii}$). Ponadto macierz $A$ musi być symetryczna oraz dodatnie określona.

In [53]:
def cholesky(A):
    n = len(A)
    L = [[0] * n for _ in range(n)]

    for i in range(n):
        for k in range(i + 1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            if i == k:
                L[i][k] = math.sqrt(A[i][i] - tmp_sum)
            else:
                L[i][k] = 1 / L[k][k] * (A[i][k] - tmp_sum)

    return L, np.array(L).transpose()  # L, U

In [54]:
def solve(A, b, L, U):
    z = np.linalg.solve(L, b)
    x = np.linalg.solve(U, z)
    return x


def random_matrix(low, high, n):  # returns random symmetric and positive matrix
    A = np.random.randint(low, high, size=(n, n))
    return np.dot(A, A.transpose()), np.random.randint(low, high, size=(n, 1))


def compare(A, b):
    L1, U1 = crout(A)
    L2, U2 = doolittle(A)
    L3, U3 = cholesky(A)

    print(f"\nCrout:\nL:")
    for i in range(len(L1)):
        print(L1[i])
    print(f"U:")
    
    for i in range(len(U1)):
        print(U1[i])
    print(f"\nDoolittle:\nL:")
    
    for i in range(len(L2)):
        print(L2[i])
    print(f"U:")
    for i in range(len(U2)):
        print(U2[i])

    print(f"\nCholesky:\nL:")
    for i in range(len(L3)):
        print(L3[i])
    print(f"U:")
    for i in range(len(U3)):
        print(U3[i])

    print(f"\nSolve using Crout:\n {solve(A, b, L1, U1)}")
    print(f"Solve using Doolittle:\n {solve(A, b, L2, U2)}")
    print(f"Solve using Cholesky:\n {solve(A, b, L3, U3)}")

In [55]:
A, b = random_matrix(3, 10, 5)
compare(A, b)


Crout:
L:
[108.0, 0, 0, 0, 0]
[138.0, 66.66666666666669, 0, 0, 0]
[108.0, 16.0, 23.16, 0, 0]
[115.0, 44.05555555555557, 1.4266666666666659, 3.345033342928421, 0]
[153.0, 55.50000000000003, 11.679999999999996, -2.3124100460564367, 1.557257108845388]
U:
[1, 1.2777777777777777, 1.0, 1.0648148148148149, 1.4166666666666667]
[0, 1, 0.23999999999999994, 0.6608333333333329, 0.8324999999999998]
[0, 0, 1, 0.061600460564191405, 0.5043177892918826]
[0, 0, 0, 1, -0.6912965608982641]
[0, 0, 0, 0, 1]

Doolittle:
L:
[1.0, 0, 0, 0, 0]
[1.2777777777777777, 1.0, 0, 0, 0]
[1.0, 0.23999999999999994, 1.0, 0, 0]
[1.0648148148148149, 0.6608333333333329, 0.06160046056419157, 1.0, 0]
[1.4166666666666667, 0.8324999999999998, 0.504317789291883, -0.6912965608982652, 1.0]
U:
[108, 138, 108, 115, 153]
[0, 66.66666666666669, 16.0, 44.05555555555557, 55.50000000000003]
[0, 0, 23.159999999999997, 1.4266666666666623, 11.680000000000007]
[0, 0, 0, 3.345033342928417, -2.3124100460564136]
[0, 0, 0, 0, 1.5572571088453628]


Jak widać na powyższym przykładzie, wszystkie 3 metody działają poprawnie. Następnym krokiem będzie pomiar czasów działania samych faktoryzacji (poza środowiskiem Jupyter). Funckja oraz czasy działania widoczne są poniżej, z kolei wykres oraz pomiary załączone są w osobnym pliku.

In [56]:
def compare_times():
    tab1 = []
    tab2 = []
    tab3 = []
    x_axis = []
    for n in range(10, 100):
        A, b = random_matrix(-1000, 1000, n)
        start_time_1 = time.time()
        crout(A)
        stop_time_1 = time.time()
        start_time_2 = time.time()
        doolittle(A)
        stop_time_2 = time.time()
        start_time_3 = time.time()
        cholesky(A)
        stop_time_3 = time.time()
        t1 = stop_time_1 - start_time_1
        t2 = stop_time_2 - start_time_2
        t3 = stop_time_3 - start_time_3
        print(f"\nMatrix {n}x{n}:")
        print(f"    Crout: {t1}s")
        print(f"    Doolittle: {t2}s")
        print(f"    Cholesky: {t3}s")
        tab1.append(t1)
        tab2.append(t2)
        tab3.append(t3)
        x_axis.append(n)

    plt.plot(x_axis, tab1, 'b.', label="Crout")
    plt.plot(x_axis, tab2, 'r.', label="Doolittle")
    plt.plot(x_axis, tab3, 'g.', label="Cholesky")
    plt.xlabel("Square matrix size")
    plt.ylabel("Time [s]")
    plt.title("Time of evaluation fro Gaussian elimination")
    plt.legend()
    plt.grid()
    plt.show()

## Zadanie 3
Zapoznaj się z funkcją rozwiązywania układów równań liniowych dostarczoną przez bibliotekę numpy/scipy. Porównaj jej wydajność z własnymi implementacjami.

Pomiary przeprowadziłem przy użcyiu poniższego kodu. Wyniki pomiarów oraz wykres znajdują się w osobnym pliku.

In [57]:
from scipy import linalg

def solve_time_comparison():
    tab1 = []
    tab2 = []
    tab3 = []
    tab4 = []
    tab5 = []
    x_axis = []
    for n in range(10, 150):
        A, b = random_matrix(-1000, 1000, n)
        start_time_1 = time.time()
        linalg.solve(A, b, crout(A))
        stop_time_1 = time.time()
        start_time_2 = time.time()
        linalg.solve(A, b, doolittle(A))
        stop_time_2 = time.time()
        start_time_3 = time.time()
        linalg.solve(A, b, cholesky(A))
        stop_time_3 = time.time()
        start_time_4 = time.time()
        np.linalg.solve(A, b)
        stop_time_4 = time.time()
        start_time_5 = time.time()
        linalg.solve(A, b)
        stop_time_5 = time.time()
        t1 = stop_time_1 - start_time_1
        t2 = stop_time_2 - start_time_2
        t3 = stop_time_3 - start_time_3
        t4 = stop_time_4 - start_time_4
        t5 = stop_time_5 - start_time_5
        print(f"\nMatrix {n}x{n}:")
        print(f"    Crout:     {t1}s")
        print(f"    Doolittle: {t2}s")
        print(f"    Cholesky:  {t3}s")
        print(f"    Numpy:     {t3}s")
        print(f"    Scipy:     {t3}s")
        tab1.append(t1)
        tab2.append(t2)
        tab3.append(t3)
        tab4.append(t4)
        tab5.append(t5)
        x_axis.append(n)

    plt.plot(x_axis, tab1, 'b.', label="Crout")
    plt.plot(x_axis, tab2, 'r.', label="Doolittle")
    plt.plot(x_axis, tab3, 'g.', label="Cholesky")
    plt.plot(x_axis, tab4, 'k.', label="Numpy")
    plt.plot(x_axis, tab5, 'y.', label="Scipy")
    plt.xlabel("Square matrix size")
    plt.ylabel("Time [s]")
    plt.title("Time of evaluation for LU decompositions")
    plt.legend()
    plt.grid()
    plt.show()